# POC Power BI TS Formation

In [685]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import os
from os import path
import glob
import xlsxwriter
import openpyxl # 3.0
import re

## 1) Déclaration de la classe Datamanagement et des fonctions associées

In [686]:
class Datamanagement:
    def __init__(self):
        self.source = {} # Dictionnaire contenant le nom du fichier source et son contenu stocké dans un DataFrame
        self.dim = {} # Dictionnaire contenant le nom et les dataframes des tables de dimension
        self.fact = {} # Dictionnaire contenant le nom et les dataframes liés aux tables de faits
        self.changed_keys = {}        
        self.group_col = pd.DataFrame(columns=["Nom colonne", "Dimension cible", "Colonne cible"], data=[]) # DataFrame contenant les colonnes de regroupement créés
        try:
            self.settings = pd.ExcelFile(r"Settings & documentation\Settings.xlsx")  # Fichier setting dans un dataframe
            self.setting_sheets = self.settings.sheet_names # List des onglets de setting
        except:
            pass


    def import_csv(self, filename, engine_val=None, encoding_val='utf-8', sep_val=';',low_memory_val=False):
        self.source[filename] = pd.read_csv(r'Data/'+ filename + '.csv', engine=engine_val, encoding=encoding_val, sep=sep_val, low_memory=low_memory_val)
        return self.source[filename]    
    
    def update_key(self, changed_keys, filename):
        self.changed_keys[filename] = changed_keys
        self.source[filename].rename(columns=changed_keys , inplace=True)
        return self.source[filename]
    
    def import_dim(self,dimname, dimdataframe):
        self.dim[dimname]=dimdataframe
        
    def import_fact(self,factname, factdataframe):
        self.fact[factname] = factdataframe
    
    def export(self):
        for key, value in self.dim.items():
            location = "Transformed data"
            file_name = str(key) + '.csv'
            location = os.path.join(location, file_name)  
            value.to_csv(location, encoding='utf-8', index=False)
            
        for key, value in self.fact.items():
            location = "Transformed data"
            file_name = str(key) + '.csv'
            location = os.path.join(location, file_name)  
            value.to_csv(location, encoding='utf-16', index=False)
            
    def import_data(self):
# Fonction générant le fichier setting lors du premier chargement si ce dernier n'existe pas
# Etape 1 lecture des fichiers présents dans data et extraction des colonnes ainsi que des informations liées à la qualité de données
# Etape 2 vérification si le fichier Settings.xlsx existe, s'il existe, il faut éventuellement le modifier avec les nouvelles informations, sinon le créer
        
        df_files_col = pd.DataFrame()
        p = 1

        #Etape 1 lecture des fichiers présents dans data et extraction des colonnes ainsi que des informations liées à la qualité de données
        for key, value in self.source.items():
            df = value

            for i, col in enumerate(df.columns):
                df_files_col.at[p, 'Nom fichier source'] = key
                df_files_col.at[p,'Nom champ source'] = col
                df_files_col.at[p,'Type'] = str(df[col].dtypes)
                df_files_col.at[p,'Synthèse'] = str(df[col].describe())
                df_files_col.at[p, "Date ajout"] = str(datetime.date(datetime.now()))
                df_files_col.at[p, "Date modification"] = str(datetime.date(datetime.now()))
                p = p + i

        df_dir_files = df_files_col.copy()
        df_dir_files = df_dir_files[["Nom fichier source", "Date ajout", "Date modification"]].drop_duplicates()

        if path.exists("Settings & documentation\Settings.xlsx"):
            pass

        else:
            writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='xlsxwriter')
            workbook  = writer.book
            df_dir_files.to_excel(writer, sheet_name='Fichiers Source', index=False)

            df_files_col[['Nom fichier source', 'Nom champ source', 'Type', 'Synthèse']].to_excel(writer, sheet_name='Fichiers et colonnes source', index=False)

            worksheet_FCS = writer.sheets["Fichiers et colonnes source"]
            worksheet_FS = writer.sheets["Fichiers Source"]
            #worksheet1 = workbook.add_worksheet("Mapping données")

            cell_format_FCS = workbook.add_format() 
            cell_format_FCS.set_text_wrap()
            cell_format_FCS.set_align('center')
            cell_format_FCS.set_align('vcenter')

            cell_format_FS = workbook.add_format()    
            cell_format_FS.set_align('left')

            worksheet_FCS.set_column('A:B', 30, cell_format_FCS)
            worksheet_FCS.set_column('C:C', 15, cell_format_FCS)
            worksheet_FCS.set_column('D:D', 25, cell_format_FCS)
            worksheet_FS.set_column('A:A', 40, cell_format_FS)
            worksheet_FS.set_column('B:B', 20, cell_format_FS)
            worksheet_FS.set_column('C:C', 20, cell_format_FS)
            writer.save()
            workbook.close()
        self.settings = pd.ExcelFile(r"Settings & documentation\Settings.xlsx")  # Fichier setting dans un dataframe
        self.setting_sheets = self.settings.sheet_names # List des onglets de setting
    def add_sheet_mapcol(self,DicDataframe=None):
# Fonction créant la feuille Mapping dim dans le fichier Settings
# Cette feuille contient l'ensemble des valeurs de dimension, elle permet de réaliser un mapping pour changer le nom des attributs

        if DicDataframe == None:
            DicDataframe = self.dim           
            
        df = pd.DataFrame(columns=["Nom dimension", "Nom colonne"],data=[])

        for key, value in DicDataframe.items():        

            for i, col in enumerate(value.columns):
                df.at[i, "Nom dimension"] = key
                df.at[i, "Nom colonne"] = col
                df.at[i, "Nouveau nom"] = ""
                df.at[i, "A mapper"] = "Non"                

        workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")  
        writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
        writer.book = workbook1
        df.to_excel(writer, sheet_name="Mapping dim colonne",engine='openpyxl',index=False)     
        writer.save()
        writer.close()

        workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")
        writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
        writer.book = workbook1
        workbook1["Mapping dim colonne"].column_dimensions["A"].width = 20
        workbook1["Mapping dim colonne"].column_dimensions["B"].width = 20
        workbook1["Mapping dim colonne"].column_dimensions["C"].width = 20

        writer.save()
        writer.close()
    
    def map_col(self):        
# Fonction transformant le nom des colonnes suivant le mapping effectué dans l'onglet Mapping dim colonne du fichier setting
# Entrée: Dictionnaire key: Nom dataframe  Value : Dataframe
# Sortie un dictionnaire "nom"/Dataframe avec les valeurs des colonnes actualisées suivant l'onglet de mapping "Mapping dim colonne" du fichier settings

        dict_dataframe = self.dim
        md = pd.read_excel(r"Settings & documentation\Settings.xlsx", sheet_name="Mapping dim colonne")
        mdf = md["Nom dimension"].unique()
        dic_map = {}
        
        for dim in mdf:
            map = md.loc[(md["Nom dimension"] == dim ) & (md["Nouveau nom"].notnull()== True),["Nom colonne","Nouveau nom"]]
            map.reset_index(drop=True, inplace=True)

            for i, val in enumerate(map.iterrows()):           
                dict_dataframe[dim].rename(columns={map.at[i, "Nom colonne"]:map.at[i,"Nouveau nom"]},inplace=True)

        my_dim = dict_dataframe    
    
    def export_to_settings(self, dataframe, sheetname, position = None):
# Fonction permettant d'exporter un dataframe vers un onglet de setting avec un formatage automatique des colonnes
# Paramètres: 
# dataframe: correspond au dataframe à importer,
# sheetname: correspond au nom de la feuille cible qui sera remplacées,
# position: permet de choisir la position de la feuille modifiée dans le fichier excel
   
        letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
        numbers = range(0,26,1)
        dic_letters = {} # Dictionnaire contenant les lettres de l'alphabet    
        val_max = [] # Liste contenant la taille des valeurs les plus longues

        # Création d'un dictionnaire stockant les lettres de l'alphabet et leur position afin d'alimenter les fonctions openpyxl
        for i in range(len(numbers)): 
            dic_letters[numbers[i]] = letters[i]

        # Alimentation de val_max
        for col in dataframe.columns:
            max = (dataframe[col].astype(str)).str.len().max()        
            if max != 0:
                val_max.append(int(max)+1)
            else:
                val_max.append(15)

       # Ouverture du fichier setting et création de l'onglet sheetname
    
        workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")  
        writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
        writer.book = workbook1
        list_sheet = workbook1.get_sheet_names() # Récupère la liste des feuilles

        if sheetname in list_sheet: # Si l'onglet existe déjà on va le supprimer pour le remplacer
            std = workbook1.get_sheet_by_name(sheetname)
            workbook1.remove_sheet(std)

        dataframe.to_excel(writer, sheet_name= sheetname, engine='openpyxl', index=False)     
        writer.save()
        writer.close()

        # Gère la position de la feuille

        workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")  
        writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
        writer.book = workbook1

        if position == None:        
            pos = len(list_sheet) - 1
        else:
            pos = position

        sheets = workbook1._sheets

        sheet = sheets.pop(len(list_sheet) - 1)
        sheets.insert(pos, sheet)

        writer.save()
        writer.close()

       # Réouverture du fichier setting
    
        workbook1 = openpyxl.load_workbook(r"Settings & documentation\Settings.xlsx")  
        writer = pd.ExcelWriter(r"Settings & documentation\Settings.xlsx", engine='openpyxl')
        writer.book = workbook1

        # Alimentation du fichier excel avec les données du dataframe

        for i, width in enumerate(val_max):
            if width < 11:
                workbook1[sheetname].column_dimensions[dic_letters[i]].width = 12
            else:
                workbook1[sheetname].column_dimensions[dic_letters[i]].width = width

        # Sauvegarde et fermeture du fichier excel
        
        writer.save()
        writer.close()    
    
    def mapp_data(self):
        result = pd.DataFrame()
        test=1

        try:       
            to_mapp_data = pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping données") # Lecture de la feuille Mapping données
            to_mapp_data = to_mapp_data.loc[to_mapp_data["Valeurs cible"].isna() == False]
            to_mapp_data.reset_index(drop=True, inplace=True)

            if to_mapp_data.empty == False:
                for i, val in enumerate(to_mapp_data.iterrows()):
                    self.dim[to_mapp_data.at[i,"Table"]][to_mapp_data.at[i,"Colonnes"]].replace(to_replace =to_mapp_data.at[i,"Valeurs actuelles"], 
                     value = to_mapp_data.at[i,"Valeurs cible"], inplace=True)
        except:
                mapp_col = pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping dim colonne")
                mapp_col = mapp_col.loc[mapp_col["A mapper"] == "Oui", ["Nom dimension", "Nom colonne", "Nouveau nom"]]
                mapp_col["Colonne"] = ""
                mapp_col.reset_index(inplace=True, drop=True)
                mapp_val = pd.DataFrame(columns=["Table", "Colonnes", "Valeurs actuelles", "Valeurs cible"])
                p=1
                if mapp_col.empty == False:
                    for i, val in enumerate(mapp_col.iterrows()):
                        if str(mapp_col.at[i, "Nouveau nom"]) != "nan":
                            mapp_col.at[i,"Colonne"] = mapp_col.at[i, "Nouveau nom"]
                        else:
                             mapp_col.at[i,"Colonne"] = mapp_col.at[i, "Nom colonne"]

                    for i,val in enumerate(mapp_col[["Nom dimension", "Colonne"]].iterrows()):

                        if (self.dim[mapp_col.at[i, "Nom dimension"]][mapp_col.at[i, "Colonne"]]).empty == False:
                            list_val = (self.dim[mapp_col.at[i, "Nom dimension"]][mapp_col.at[i, "Colonne"]]).unique()
                            nom_col = mapp_col.at[i, "Colonne"]
                            nom_table = mapp_col.at[i, "Nom dimension"]
                        
                        if len(list_val) != 0:
                            for i, val in enumerate(list_val):                           
                                mapp_val.at[p,"Table"] = nom_table
                                mapp_val.at[p,"Colonnes"] = nom_col
                                mapp_val.at[p,"Valeurs actuelles"] = val
                                mapp_val.at[p,"Valeurs cible"] = ""
                                p += 1

                if mapp_val.empty: 
                    pass            
                else:
                    self.export_to_settings(mapp_val,"Mapping données")
                    
    def __compare__(self,sheet_name, cible_dataframe, excl_col= [], position=None):        
        source = pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name=sheet_name)
        print(source)
        source["compare"] = ""
        cible = cible_dataframe
        cible["compare"] = ""
        colonnes_compared= [col for col in list(source.columns) if col not in excl_col]
        
        for col in colonnes_compared:
            source["compare"] = source["compare"] + source[col].astype('str')        
        
        for col in colonnes_compared:
            cible["compare"] = cible["compare"] + cible[col].astype('str')
        
        to_keep = [line for line in list(source["compare"]) if line in list(cible["compare"])]
        
        
        to_add = [line for line in list(cible["compare"]) if line not in list(source["compare"])]
        
        result = source.loc[source["compare"].isin(to_keep)]

        
        if to_add != "":
            result = result.append(cible.loc[cible["compare"].isin(to_add)])
            
        result.reset_index(drop=True, inplace=True)
        result.drop(["compare"], axis=1, inplace=True)
        return result
        #self.export_to_settings(result,sheet_name,position)
    
    def update_fichier_source(self):
        #Fonction mettant à jour l'onglet "Fichiers Source" du fichier Setting
        
        df_files_col = pd.DataFrame()
        p = 1

        #lecture des fichiers présents dans data et extraction des noms des fichiers     
        for key, value in self.source.items():
            
            df = value

            for i, col in enumerate(df.columns):
                df_files_col.at[p, 'Nom fichier source'] = key
                df_files_col.at[p,'Nom champ source'] = col
                df_files_col.at[p, "Date ajout"] = ""
                df_files_col.at[p, "Date modification"] = ""
                p = p + i

        df_dir_files = df_files_col.copy()
        df_dir_files = df_dir_files[["Nom fichier source", "Date ajout", "Date modification"]].drop_duplicates()
        result = self.__compare__("Fichiers Source", df_dir_files, ["Date ajout", "Date modification"],0 )
        result["Date ajout"] = result["Date ajout"].fillna(str(datetime.date(datetime.now())))
        result["Date modification"] = str(datetime.date(datetime.now()))
        
        self.export_to_settings(result,"Fichiers Source",0)
        
    def update_columns(self):
        #Fonction mettant à jour l'onglet Mapping dim colonne suite à l'ajout des nouvelles dimensions
        existing_colmap =  pd.read_excel(r"Settings & documentation\Settings.xlsx", sheet_name='Mapping dim colonne') # Contient les valeurs de mapping dim colonne
        fresh_col = pd.DataFrame(columns=["Nom dimension", "Nom colonne"], data=[]) # Contient l'ensemble des nouvelles valeurs pour Mapping dim colonne
        existing_colmap["Comparaison"] = existing_colmap["Nom dimension"] + existing_colmap["Nom colonne"] # Création de la colonne "Comparaison" servant à comparer les lignes
        to_keep = [] # Contient la liste des valeurs de la colonne "Comparaison" à conserver dans existing_colmap
        to_add = [] # Contient la liste des valeurs de la colonne "Comparaison" à rajouter
        p=0

        for key, value in self.dim.items():

            for i, col in enumerate(value.columns):
                fresh_col.at[p, "Nom dimension"] = key
                fresh_col.at[p, "Nom colonne"] = col
                fresh_col.at[p, "Nouveau nom"] = ""
                fresh_col.at[p, "A mapper"] = "Non"
                p+=1

        fresh_col["Comparaison"] = fresh_col["Nom dimension"] + fresh_col["Nom colonne"]
        to_keep = list(fresh_col["Comparaison"])
        existing_colmap = existing_colmap.loc[existing_colmap["Comparaison"].isin(to_keep)]
        existing_colmap.reset_index(drop=True, inplace=True)

        # On détermine la liste des nouvelles valeurs absentes de la feuille de mapping

        for val in to_keep:
            if val in list(existing_colmap["Comparaison"]):
                pass

            else:
                to_add.append(val)

        fresh_col = fresh_col.loc[fresh_col["Comparaison"].isin(to_add)]
        fresh_col.reset_index(drop=True, inplace=True)

        result = existing_colmap.append(fresh_col)

        # On exporte le résultat dans l'onglet Mapping dim colonne de Setting
        result = result.drop(["Comparaison"], axis=1)
        result = result.sort_values(["Nom dimension", "Nom colonne"]) 
        self.export_to_settings(result, "Mapping dim colonne",2)
        
        return result
    
    def update_map_val(self):       
        mapp_col = pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping dim colonne")
        mapp_col = mapp_col.loc[mapp_col["A mapper"] == "Oui", ["Nom dimension", "Nom colonne", "Nouveau nom"]]
        mapp_col["Colonne"] = ""
        mapp_col.reset_index(inplace=True, drop=True)
        mapp_val = pd.DataFrame(columns=["Table", "Colonnes", "Valeurs actuelles", "Valeurs cible"])
        p=1
        if mapp_col.empty == False:
            for i, val in enumerate(mapp_col.iterrows()):
                if str(mapp_col.at[i, "Nouveau nom"]) != "nan":
                    mapp_col.at[i,"Colonne"] = mapp_col.at[i, "Nouveau nom"]
                else:
                     mapp_col.at[i,"Colonne"] = mapp_col.at[i, "Nom colonne"]        

            for i,val in enumerate(mapp_col[["Nom dimension", "Colonne"]].iterrows()):

                if (self.dim[mapp_col.at[i, "Nom dimension"]][mapp_col.at[i, "Colonne"]]).empty == False:
                    list_val = (self.dim[mapp_col.at[i, "Nom dimension"]][mapp_col.at[i, "Colonne"]]).unique()
                    nom_col = mapp_col.at[i, "Colonne"]
                    nom_table = mapp_col.at[i, "Nom dimension"]

                if len(list_val) != 0:
                    for i, val in enumerate(list_val):                           
                        mapp_val.at[p,"Table"] = nom_table
                        mapp_val.at[p,"Colonnes"] = nom_col
                        mapp_val.at[p,"Valeurs actuelles"] = val
                        mapp_val.at[p,"Valeurs cible"] = ""
                        p += 1
                        
        if mapp_val.empty: 
            pass     
        else:
            result = self.__compare__("Mapping données", mapp_val,excl_col=["Valeurs cible"])
            self.export_to_settings(result,"Mapping données",3)
            
    def add_col_group(self, regroup_name, dim_source, col_source):
            
        target_sheet = pd.DataFrame(columns=["Dimension source", "Colonne source", "Valeurs sources", "Valeurs cibles"], data=[])
        export_name =  "Gr_" + dim_source + "_" + regroup_name 
        
        # Préparation du dataframe target_sheet
        
        for i, val in enumerate(list(self.dim[dim_source][col_source].unique())):
            target_sheet.at[i, "Dimension source"] = dim_source
            target_sheet.at[i, "Colonne source"] = col_source
            target_sheet.at[i, "Valeurs sources"] = val
        
        if export_name in self.setting_sheets:
            result = self.__compare__(export_name, target_sheet, excl_col= ["Valeurs cibles"])
            self.export_to_settings(result,export_name)
            replaced_values = list(result.loc[result["Valeurs cibles"].isna() == False, "Valeurs sources"])
            replacing_values = list(result.loc[result["Valeurs cibles"].isna() == False, "Valeurs cibles"])
            print(replaced_values)
            print(replacing_values)
            self.dim[dim_source][export_name] = self.dim[dim_source][col_source]
            self.dim[dim_source][export_name].replace(replaced_values, replacing_values, inplace=True)
        else:
            self.export_to_settings(target_sheet, export_name)
                   
        
        

### Instanciation de la classe Datamanagement

In [687]:
my_data = Datamanagement()

## 2) Import des données

In [688]:
%%time
### Ajouter les tables à intégrer ici
tcw = my_data.import_csv('191119_TrainingCollectiveWishes') # Demandes collectives non affectées à un plan de formation
tiw = my_data.import_csv('191119_TrainingIndividualWishes') # Demandes individuelles non affectées à un plan de formation
tp = my_data.import_csv('191119_TrainingPlan') # Plans de formation
tpcw = my_data.import_csv('191119_TrainingPlanCollectiveWishes') # Demande de formation collectives prises en charge dans un plan de formation
tpiw = my_data.import_csv('191119_TrainingPlanIndividualWishes') # Demande de formation individuelles prises en charge dans un plan de formation
tr = my_data.import_csv('191119_TrainingRegister') # Table recensant les inscription aux sessions de formation 
ts = my_data.import_csv('191119_TrainingSessions') # Table recensant les sessions de formation 
tsc = my_data.import_csv('191119_TrainingStageCost') # Table contenant les coûts des stages
tsv2 = my_data.import_csv('191119_TrainingStagev2') # Table contenant les stages
emp = my_data.import_csv('191119_Employees') # Table maître employé
empc = my_data.import_csv('191119_EmployementContract') # Table contrat employé
indO = my_data.import_csv('191119_IndividualOrganization') # Table organisation employé
indpp =my_data.import_csv('191119_IndivPPCsNew') # ?

Wall time: 683 ms


### Initialisation du fichier setting ou update du fichier setting

In [689]:
my_data.import_data()
my_data.update_fichier_source()

                     Nom fichier source  Date ajout Date modification
0       191119_TrainingCollectiveWishes  2020-01-14        2020-01-14
1       191119_TrainingIndividualWishes  2020-01-14        2020-01-14
2                   191119_TrainingPlan  2020-01-14        2020-01-14
3   191119_TrainingPlanCollectiveWishes  2020-01-14        2020-01-14
4   191119_TrainingPlanIndividualWishes  2020-01-14        2020-01-14
5               191119_TrainingRegister  2020-01-14        2020-01-14
6               191119_TrainingSessions  2020-01-14        2020-01-14
7              191119_TrainingStageCost  2020-01-14        2020-01-14
8                191119_TrainingStagev2  2020-01-14        2020-01-14
9                      191119_Employees  2020-01-14        2020-01-14
10           191119_EmployementContract  2020-01-14        2020-01-14
11        191119_IndividualOrganization  2020-01-14        2020-01-14
12                  191119_IndivPPCsNew  2020-01-14        2020-01-14


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:181: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:184: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:185: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).


## 3) Renommage des clés fonctionnelles

In [690]:
%%time
ts = my_data.update_key({'clientcode':'SessionID', 'coursecode':'FormationID', 'startdate':'SessionDate'},'191119_TrainingSessions')
tsv2 = my_data.update_key({'clientcode':'SessionID'},'191119_TrainingStagev2')
tr = my_data.update_key({'clientcode':'SessionID', 'traineusername':'USERNAME','trainingplanclientcode':'PlanID', 'trainingwishclientcode':'WishID'}, '191119_TrainingRegister')
tp = my_data.update_key({'plancode':'PlanID'}, '191119_TrainingPlan')
#emp = my_data.update_key({'username':'USERNAME'}, '191119_Employees')
tcw = my_data.update_key({'clientcode':'WishID','coursecode':'FormationID','username':'USERNAME'}, '191119_TrainingCollectiveWishes')
tiw = my_data.update_key({'username':'USERNAME', 'employeenumber':'ZY00.MATCLE', 'clientcode':'WishID', 'coursecode':'FormationID'},'191119_TrainingIndividualWishes')
tpcw = my_data.update_key({'plan_code':'PlanID', 'wish_code':'WishID'}, '191119_TrainingPlanCollectiveWishes')
tpiw = my_data.update_key({'plan_code':'PlanID', 'wish_code':'WishID'} ,'191119_TrainingPlanIndividualWishes')

Wall time: 20 ms


## 5) Creation des tables de dimension

### Création de la table dimension demandes de formations

In [691]:
%%time
# Génération de la table de dimension Souhaits de formation à partir des tables 191119_TrainingCollectiveWishes et 191119_TrainingIndividualWishes 
dim_wish = tcw.append(tiw,sort=True)
dim_wish.reset_index(drop=True, inplace=True)
dim_wish["Wish_key"] = dim_wish.index
dim_wish.loc[dim_wish.nbtrainees.isna(), "nbtrainees"] = 1
dim_wish.index.name="key_wish"
my_data.import_dim("Souhaits de formation", dim_wish)

Wall time: 60 ms


### Création de la table dimension plan de formation

In [692]:
%%time
# Génération de la table plan de formation
dim_plan = tp

# On ajoute une clé technique à la table dim_plan
dim_plan["key_plan"] = dim_plan.index

# On insère une ligne pour affecter par la suite les demandes non affectées à une clé plan
empty_data = {"PlanID" : ["Pas de plan"], "key_plan" : [9999]}
empty_plan = pd.DataFrame(data=empty_data)
dim_plan= dim_plan.append(empty_plan,ignore_index=True)

my_data.import_dim("Plan de formation", dim_plan)


Wall time: 10 ms


### Traitement du mapping des colonnes et des valeurs

In [693]:
%%time

#Verification si la colonne Mapping dim existe dans le fichier setting, si non on va la créer
try:
    pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping dim colonne")   
    Mapp_sheet = True    
except:
    Mapp_sheet = False
    
    
if Mapp_sheet == True:
    my_data.update_columns()
    pass
else:    
    #add_sheet_mapcol(my_data.dim)
    my_data.add_sheet_mapcol()

my_data.map_col()
#my_data.dim = map_col(my_data.dim)

my_data.mapp_data()

try:
    pd.read_excel(r'Settings & documentation\Settings.xlsx', sheet_name="Mapping données")   
    Mapp_val = True    
except:
    Mapp_val = False
    
if Mapp_val == True :
    my_data.update_map_val()

my_data.add_col_group("group1", "Plan de formation", "planname")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:181: DeprecationWarning: Call to deprecated function get_sheet_names (Use wb.sheetnames).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:184: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:185: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).


      Dimension source Colonne source        Valeurs sources  Valeurs cibles
0    Plan de formation       planname             18_DEFISOL             NaN
1    Plan de formation       planname   20_BIOLINE_CORPORATE             NaN
2    Plan de formation       planname  20_UNION_INVIVO_SILOS             NaN
3    Plan de formation       planname           20_PHYTEUROP             NaN
4    Plan de formation       planname      20_SICA_DE_GOUAIX             NaN
..                 ...            ...                    ...             ...
127  Plan de formation       planname  18_SEMENCES_DE_FRANCE             NaN
128  Plan de formation       planname             18_LOGITIA             NaN
129  Plan de formation       planname      18_SICA_DE_GOUAIX             NaN
130  Plan de formation       planname       18_LS_PRODUCTION             NaN
131  Plan de formation       planname                    NaN             NaN

[132 rows x 4 columns]
[]
[]
Wall time: 2.13 s


## 6) Création des tables de fait

### Table de fait souhaits de formation

In [694]:
%%time
# Création des indicateurs de fomation prévisionnelle, ces derniers s'appuient sur les 4 tables liées aux souhaits de formation
#Grain = 1 ligne correspond à un souhait de formation

# Les tables contenant l'ensemble des souhaits de formation 191119_TrainingCollectiveWishes et 191119_TrainingIndividualWishes vont être fusionnées dans la table wish_all
# Seules les clés externes et les attributs servant au calcule des indicateurs seront conservés
# La table 191119_TrainingIndividualWishes ne contenant pas l'indicateur "nbtrainees", nous considérons qu'une ligne équivaut à 1 nbtrainees

tcw_staging = tcw.copy()
tcw_staging = tcw_staging[["WishID","employeenumber", "FormationID", "nbtrainees", "nbmen", "nbwomen"]]
tiw_staging = tiw.copy()
tiw_staging = tiw_staging[["WishID", "ZY00.MATCLE","FormationID"]]
tiw_staging["nbtrainees"]=1

wish_all = tcw_staging.append(tiw_staging)

# Les tables contenant les demandes de formation associées à un plan vont être fusionnées
# Etape 1 récupérer les tables 191119_TrainingPlanCollectiveWishes et 191119_TrainingPlanIndividualWishes correspondant aux souhaits de formation associés à des plans
# Etape 2 on va supprimer les champs descriptifs: action, default_currency,hourly_wage_rage
# Etape 3 on fusionne les tables dans la table wish_plan

tpcw_staging = tpcw.copy()
tpcw_staging = tpcw_staging.drop(columns=["action", "default_currency","hourly_wage_rate", "training_system"])
tpiw_staging = tpiw.copy()
tpiw_staging = tpiw_staging.drop(columns=["action", "default_currency", "training_system"])

wish_plan = tpcw_staging.copy()
wish_plan = wish_plan.append(tpiw_staging)

# On fusionne les 2 tables créées afin de créer la table de fait wish_fact

wish_fact = pd.merge(wish_all, wish_plan, on = "WishID", how='left')
wish_fact.reset_index(drop=True, inplace=True)

# On ajoute la clé technique de la table plan de formation

wish_fact = pd.merge(wish_fact, dim_plan[['PlanID','key_plan']], on='PlanID', how='left')

# On ajoute l'indicateur in_plan
wish_fact["In_plan"] = ""
wish_fact.loc[wish_fact['key_plan'].isna() == True,"In_plan"] = 0
wish_fact.loc[wish_fact['key_plan'].isna() == False,"In_plan"] = 1

# Typage des valeurs
wish_fact["wage_cost"] = wish_fact["wage_cost"].str.replace(",",".", regex=True)

wish_fact["wage_cost"] = wish_fact["wage_cost"].astype('float64')
wish_fact["key_plan"] = wish_fact["key_plan"].fillna(9999)
wish_fact["key_plan"] = wish_fact["key_plan"].astype('int64')


# Stockage dans my_data de la table wish_fact 

my_data.import_fact("Indicateurs Prévisonnels", wish_fact)


Wall time: 176 ms


### Table de faits des formations suivi du plan de formation

In [695]:
#Préparation de la table de fait contenant les indicateurs liés au suivi du plan
tr_staging = tr.copy()

## Export des données vers le répertoire Transformed data

In [696]:
%%time

my_data.export()

Wall time: 1.14 s


In [697]:
print(my_data.dim["Plan de formation"].columns)

Index(['PlanID', 'beginexecutiondate', 'endexecutiondate', 'initialbudget',
       'initialnumberofhours', 'key_plan', 'planname', 'state',
       'validationdate', 'Gr_Plan de formation_group1'],
      dtype='object')


['18_DEFISOL' '20_BIOLINE_CORPORATE' '20_UNION_INVIVO_SILOS'
 '20_PHYTEUROP' '20_SICA_DE_GOUAIX' '20_SICA_SILO_PORTUAIRE_DE_BORDEAUX'
 '20_SILO_HUNINGUE' '20_UNION_INVIVO_AGRO' '20_AGRINOVEX'
 '20_AGROSOLUTIONS' '20_BE_API' '20_BIOLINE_INSURANCE'
 '20_DEFISOL_SERVICES' '20_SMAG' '20_SEMENCES_DE_FRANCE'
 '20_BIOLINE_FRANCE' '20_LS_PRODUCTION' '20_INVIVO_DIGITAL_FACTORY'
 '2020-NEODIS' '2020-INVIVO GROUP' '2020-GAMM VERT' '2020-INVIVO TRADING'
 '2020-INVIVO EVENTS' '2020- FOOD AND TECH' '2020-OUIFIELD'
 '2020- GAMM VERT SUD OUEST' "2020- FRAIS D'ICI" "2018_FRAIS D'ICI"
 '2018-FOOD & TECH' '2018-QALIAN' '2018-UPSCIENCE' '2018-ADGENE'
 '2018-SANICOOPA' '2018-EVIALIS FRANCE' '2018-NUTRILAC' '2018-SERMIX'
 '2018-MERIEL' '2018-NEOVIA' '2018-INVIVO MANAGEMENT' 'TEST' '2019-SERMIX'
 '2019-NEOVIA' '2019-ADGENE' '2019 - GAMM VERT' '2019-OUIFIELD'
 "2019-FRAIS D'ICI" '2019-NUTRILAC' '2019-NEODIS' '2019-EVIALIS FRANCE'
 '2019-UPSCIENCE' '2019-SANICOOPA' '2019-GAMM VERT SUD OUEST'
 '2019-INVIVO EVEN